# CellRank Basics

This tutorial introduces you to CellRank’s high level API for computing initial & terminal states and fate probabilities. Once we have the fate probabilities, this tutorial shows you how to use them to plot a directed [Wolf et al., 2019], to compute putative lineage drivers and to visualize smooth gene expression trends. If you want a bit more control over how initial & terminal states and fate probabilities are computed, then you should check out CellRank’s low level API, composed of kernels and estimators. This really isn’t any more complicated than using scikit-learn, so please do check out the Kernels and estimators tutorial.

In this tutorial, we will use RNA velocity and transcriptomic similarity to estimate cell-cell transition probabilities. Using kernels and estimators, you can apply CellRank even without RNA velocity information, check out our CellRank beyond RNA velocity tutorial. CellRank generalizes beyond RNA velocity and is a widely applicable framework to model single-cell data based on the powerful concept of Markov chains.

The first part of this tutorial is very similar to scVelo’s tutorial on pancreatic endocrinogenesis. The data we use here comes from [Bastidas-Ponce et al., 2019]. For more info on scVelo, see the documentation or take a look at [Bergen et al., 2020].

This tutorial notebook can be downloaded using the following link.

# Import Packages and Data

Easiest way to start is to download Miniconda3 along with the environment file found here. To create the environment, run conda create -f environment.yml.

In [2]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import os.path as op
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, "/Users/geenaildefonso/Projects/")
import mazebox as mb
sys.path.insert(1, '/Users/geenaildefonso/Dropbox (VU Basic Sciences)/')
from typing import Any
from copy import copy
from anndata import AnnData
import scipy.sparse as sp

In [3]:
import warnings

warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

In [4]:
adata = scv.read('/Users/geenaildefonso/Dropbox (VU Basic Sciences)/RPM/adata05.h5ad')

In [34]:
adata

AnnData object with n_obs × n_vars = 15257 × 21163
    obs: 'Clusters', '_X', '_Y', 'dropkick_score', 'dropkick_label', 'arcsinh_n_genes_by_counts', 'batch', 'timepoint', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'total_counts_ambient', 'log1p_total_counts_ambient', 'pct_counts_ambient', 'arcsinh_total_counts', 'S_score', 'G2M_score', 'phase', 'leiden', 'barcode', 'kept', 'n_counts', 'velocity_self_transition', 'velocity_length', 'velocity_confidence', 'velocity_confidence_transition', 'root_cells', 'end_points', 'velocity_pseudotime', 'dpt_pseudotime', 'col', 'absorbing', 'absorb_group', 'log1p_plasticity', 'SCLC-Y_Score', 'SCLC-Y_Col', 'SCLC-P_Score', 'SCLC-P_Col', 'SCLC-N_Score', 'SCLC-N_Col', 'SCLC-A2_Score', 'SCLC-A2_Col', 'SCLC-A_Score', 'SCLC-A_Col', 'Phenotype_unfiltered', 'max', 'SCLC-Yscore', 'SCLC-Yco

In [26]:
from cellrank.tl.kernels import VelocityKernel

vk = VelocityKernel(adata)

In [27]:
vk.compute_transition_matrix()

  0%|          | 0/15257 [00:00<?, ?cell/s]

  0%|          | 0/15257 [00:00<?, ?cell/s]

<VelocityKernel>

In [29]:
ptk = PTKernel(adata).compute_transition_matrix()

Reading `adata.obs['barcode']`


In [30]:
combined_kernel = 0.8 * vk + 0.2 * ptk

In [31]:
print(combined_kernel)

((0.8 * <VelocityKernel[softmax_scale=4.78, mode=deterministic, seed=2732, scheme=<CorrelationScheme>]>) + (0.2 * <PTKernel[some_parameter=0.5]>))


In [5]:
pseudo = pd.read_csv('PseudoCorr.csv', index_col=0, header = 0)

In [6]:
pseudo.reset_index(inplace=True)
pseudo = pseudo.rename(columns = {'index':'Barcode'})

In [ ]:
adata.obs

In [7]:
pseudo.head()

,Barcode,Diffusion Pseudotime,Monocle Pseudotime,time
0,AAAGGTAGTTGCAAGG_15806X1,0.080624,7.026657,4
1,AACCACATCCACCTCA_15806X1,0.070802,4.444920,4
2,AAGCGAGAGACCAGAC_15806X1,0.069488,2.045337,4
3,AAGTACCGTTCGTACA_15806X1,0.057792,3.756390,4
4,AAGTCGTTCGTAGGGA_15806X1,0.069608,1.996313,4


In [8]:
pseudo.columns

Index(['Barcode', 'Diffusion Pseudotime', 'Monocle Pseudotime', 'time'], dtype='object')

In [55]:
# bcs = pd.read_csv('../data/RPM_cell_barcodes.csv', index_col=0, header = 0)
# bcs['barcode_short'] = [i.split('_')[0] for i in bcs['Tumor Cell Barcodes']]

# adata.obs['barcode'] = [str(i.split(":")[1].split('x')[0]) for i in adata.obs_names]

# filtered_cells = []
# for i in adata.obs['barcode']:
#     if i in list(bcs['barcode_short']): 
#         filtered_cells.append(True)
#     else:
#         filtered_cells.append(False)
# adata.obs['kept'] = filtered_cells
# sc.pl.umap(adata, color = 'kept')

# pseudo = pd.read_csv('PseudoCorr.csv', index_col=0, header = 0)
pseudo['barcode'] = [i.split('_')[0] for i in pseudo['Barcode']]

adata.obs['barcode'] = [str(i.split(":")[1].split('x')[0]) for i in adata.obs_names]

filtered_cells = []
for i in adata.obs['barcode']:
    if i in list(pseudo['barcode']): 
        filtered_cells.append(True)
    else:
        filtered_cells.append(False)
adata.obs['kept'] = filtered_cells
# sc.pl.umap(adata, color = 'kept')

In [57]:
print(len(filtered_cells))
print(sum(filtered_cells)) #ones that are true 

15257
15110


In [56]:
pseudo.head()

,Barcode,Diffusion Pseudotime,Monocle Pseudotime,time,barcode_short,barcode
0,AAAGGTAGTTGCAAGG_15806X1,0.080624,7.026657,4,AAAGGTAGTTGCAAGG,AAAGGTAGTTGCAAGG
1,AACCACATCCACCTCA_15806X1,0.070802,4.444920,4,AACCACATCCACCTCA,AACCACATCCACCTCA
2,AAGCGAGAGACCAGAC_15806X1,0.069488,2.045337,4,AAGCGAGAGACCAGAC,AAGCGAGAGACCAGAC
3,AAGTACCGTTCGTACA_15806X1,0.057792,3.756390,4,AAGTACCGTTCGTACA,AAGTACCGTTCGTACA
4,AAGTCGTTCGTAGGGA_15806X1,0.069608,1.996313,4,AAGTCGTTCGTAGGGA,AAGTCGTTCGTAGGGA


In [52]:
df1 = adata.obs
df2 = pseudo

In [53]:
print(df1.columns)
print(df2.columns)

Index(['Clusters', '_X', '_Y', 'dropkick_score', 'dropkick_label',
       'arcsinh_n_genes_by_counts', 'batch', 'timepoint',
       'initial_size_spliced', 'initial_size_unspliced',
       ...
       'SCLC-P_Sig_Score', 'SCLC-P_Z_Score', 'SCLC-Y_Sig_Score',
       'SCLC-Y_Z_Score', 'Phenotype', 'SCLC-A_Score_pos', 'SCLC-A2_Score_pos',
       'SCLC-N_Score_pos', 'SCLC-P_Score_pos', 'SCLC-Y_Score_pos'],
      dtype='object', length=107)
Index(['Barcode', 'Diffusion Pseudotime', 'Monocle Pseudotime', 'time',
       'barcode_short'],
      dtype='object')


In [54]:
df3 = df1.where(df1['barcode'].values==df2['barcode_short'].values)

ValueError: Array conditional must be same shape as self

In [47]:
pseudo['barcode_short']

0          AAAGGTAGTTGCAAGG
1          AACCACATCCACCTCA
2          AAGCGAGAGACCAGAC
3          AAGTACCGTTCGTACA
4          AAGTCGTTCGTAGGGA
                ...        
17058    TTATTGCCAACTTCTT-1
17059    TTCCTCTTCCTGGGTG-1
17060    TTCGCTGGTCCGACGT-1
17061    TTCGGTCGTCTGTCCT-1
17062    TTTCATGTCATCACTT-1
Name: barcode_short, Length: 17063, dtype: object

In [46]:
adata.obs['barcode']

SRR11594440:AACCACATCCACCTCAx-0-day_4    AACCACATCCACCTCA
SRR11594440:AAAGGTAGTTGCAAGGx-0-day_4    AAAGGTAGTTGCAAGG
SRR11594440:AAGCGAGAGACCAGACx-0-day_4    AAGCGAGAGACCAGAC
SRR11594440:AAGTCGTTCGTAGGGAx-0-day_4    AAGTCGTTCGTAGGGA
SRR11594440:AAGTACCGTTCGTACAx-0-day_4    AAGTACCGTTCGTACA
                                               ...       
SRR11594446:TTTGACTGTACCTTCCx-day_21     TTTGACTGTACCTTCC
SRR11594446:TTTGGAGGTGGCTACCx-day_21     TTTGGAGGTGGCTACC
SRR11594446:TTTGATCAGACTTCGTx-day_21     TTTGATCAGACTTCGT
SRR11594446:TTTGGTTAGGAAGTGAx-day_21     TTTGGTTAGGAAGTGA
SRR11594446:TTTGGAGCACCAACATx-day_21     TTTGGAGCACCAACAT
Name: barcode, Length: 15257, dtype: object

In [ ]:
class MyKernel(cr.tl.kernels.Kernel):
    def compute_transition_matrix(self, some_parameter: float = 0.5) -> "MyKernel":
        transition_matrix = sp.diags(
            (some_parameter,) * len(self.adata), dtype=np.float64
        )
        self._compute_transition_matrix(transition_matrix, density_normalize=True)
        return self

    def copy(self) -> "MyKernel":
        return copy(self)

In [28]:
class PTKernel(cr.tl.kernels.Kernel):
    def __init__(
        self, adata: adata, obs_key: str = "barcode", **kwargs: Any):
        super().__init__(adata=adata, obs_key=obs_key, **kwargs)

    def _read_from_adata(self, obs_key: str, **kwargs: Any):
        super()._read_from_adata(**kwargs)

        print(f"Reading `adata.obs[{obs_key!r}]`")
        self.pseudotime = self.adata.obs[obs_key].values

    def compute_transition_matrix(self, some_parameter: float = 0.5):
        if self._reuse_cache({"some_parameter": some_parameter}):
            print("Using cached values for parameters:", self.params)
            return self

        transition_matrix = sp.diags(
            (some_parameter,) * len(self.adata), dtype=np.float64
        )

        self._compute_transition_matrix(transition_matrix, density_normalize=True)

        return self

    def copy(self):
        return copy(self)

In [10]:
class PTKernel(cr.tl.kernels.Kernel):
    def __init__(
        self, adata: adata, obs_key: str = "barcode", **kwargs: Any):
        super().__init__(adata=adata, obs_key=obs_key, **kwargs)

    def _read_from_adata(self, obs_key: str, **kwargs: Any):
        super()._read_from_adata(**kwargs)

        print(f"Reading `adata.obs[{obs_key!r}]`")
        self.pseudotime = self.adata.obs[obs_key].values

    def compute_transition_matrix(self, some_parameter: float = 0.5):
        if self._reuse_cache({"some_parameter": some_parameter}):
            print("Using cached values for parameters:", self.params)
            return self

        transition_matrix = sp.diags(
            (some_parameter,) * len(self.adata), dtype=np.float64
        )

        self._compute_transition_matrix(transition_matrix, density_normalize=True)

        return self

    def copy(self):
        return copy(self)

In [12]:
pseudo.head()

,Barcode,Diffusion Pseudotime,Monocle Pseudotime,time,barcode_short
0,AAAGGTAGTTGCAAGG_15806X1,0.080624,7.026657,4,AAAGGTAGTTGCAAGG
1,AACCACATCCACCTCA_15806X1,0.070802,4.444920,4,AACCACATCCACCTCA
2,AAGCGAGAGACCAGAC_15806X1,0.069488,2.045337,4,AAGCGAGAGACCAGAC
3,AAGTACCGTTCGTACA_15806X1,0.057792,3.756390,4,AAGTACCGTTCGTACA
4,AAGTCGTTCGTAGGGA_15806X1,0.069608,1.996313,4,AAGTCGTTCGTAGGGA


In [11]:
k = PTKernel(adata).compute_transition_matrix()
k

Reading `adata.obs['barcode_short']`


KeyError: 'barcode_short'

In [58]:
bcs = pd.read_csv('/Users/geenaildefonso/Dropbox (VU Basic Sciences)/data/RPM_cell_barcodes.csv', index_col=0, header = 0)

In [60]:
bcs.head()

,Tumor Cell Barcodes,Day or Tumor Number,Gnomex ID (matches GEO deposit)
1,AAAGGTAGTTGCAAGG_15806X1,4,15806X1
2,AACCACATCCACCTCA_15806X1,4,15806X1
3,AAGCGAGAGACCAGAC_15806X1,4,15806X1
4,AAGTACCGTTCGTACA_15806X1,4,15806X1
5,AAGTCGTTCGTAGGGA_15806X1,4,15806X1


In [ ]:
bcs = pd.read_csv('../data/RPM_cell_barcodes.csv', index_col=0, header = 0)
bcs['barcode_short'] = [i.split('_')[0] for i in bcs['Tumor Cell Barcodes']]

adata.obs['barcode'] = [str(i.split(":")[1].split('x')[0]) for i in adata.obs_names]

filtered_cells = []
for i in adata.obs['barcode']:
    if i in list(bcs['barcode_short']): 
        filtered_cells.append(True)
    else:
        filtered_cells.append(False)
adata.obs['kept'] = filtered_cells
sc.pl.umap(adata, color = 'kept')

In [40]:
df = adata.obs

In [42]:
print(df['barcode'])

SRR11594440:AACCACATCCACCTCAx-0-day_4    AACCACATCCACCTCA
SRR11594440:AAAGGTAGTTGCAAGGx-0-day_4    AAAGGTAGTTGCAAGG
SRR11594440:AAGCGAGAGACCAGACx-0-day_4    AAGCGAGAGACCAGAC
SRR11594440:AAGTCGTTCGTAGGGAx-0-day_4    AAGTCGTTCGTAGGGA
SRR11594440:AAGTACCGTTCGTACAx-0-day_4    AAGTACCGTTCGTACA
                                               ...       
SRR11594446:TTTGACTGTACCTTCCx-day_21     TTTGACTGTACCTTCC
SRR11594446:TTTGGAGGTGGCTACCx-day_21     TTTGGAGGTGGCTACC
SRR11594446:TTTGATCAGACTTCGTx-day_21     TTTGATCAGACTTCGT
SRR11594446:TTTGGTTAGGAAGTGAx-day_21     TTTGGTTAGGAAGTGA
SRR11594446:TTTGGAGCACCAACATx-day_21     TTTGGAGCACCAACAT
Name: barcode, Length: 15257, dtype: category
Categories (15201, object): ['AAACCCAAGCCGAATG', 'AAACCCAAGCGGTATG', 'AAACCCAAGGGACCAT', 'AAACCCAAGTGTTGTC', ..., 'TTTGTTGGTCCGAAGA', 'TTTGTTGGTCTTGTCC', 'TTTGTTGGTGCCGTAC', 'TTTGTTGTCCCAAGTA']


In [21]:
print(adata.shape)
print(pseudo.shape)

(15257, 21163)
(17063, 4)


In [35]:
pseudo = pd.read_csv('PseudoCorr.csv', index_col=0, header = 0)
print(pseudo[:15138])

                          Diffusion Pseudotime  Monocle Pseudotime time
Barcode                                                                
AAAGGTAGTTGCAAGG_15806X1              0.080624            7.026657    4
AACCACATCCACCTCA_15806X1              0.070802            4.444920    4
AAGCGAGAGACCAGAC_15806X1              0.069488            2.045337    4
AAGTACCGTTCGTACA_15806X1              0.057792            3.756390    4
AAGTCGTTCGTAGGGA_15806X1              0.069608            1.996313    4
...                                        ...                 ...  ...
TTTGGTTCAAGATTGA_15862X1              0.118453           16.941730   21
TTTGGTTCACGAAAGC_15862X1              0.122645           17.632420   21
TTTGTTGAGCGACTGA_15862X1              0.134370           20.864000   21
TTTGTTGGTCTTGTCC_15862X1              0.126518           17.940260   21
TTTGTTGTCCCAAGTA_15862X1              0.106923           17.002460   21

[15138 rows x 3 columns]


In [10]:
pseudo['Diffusion Pseudotime']

0        0.080624
1        0.070802
2        0.069488
3        0.057792
4        0.069608
           ...   
17058    0.100759
17059    0.125701
17060    0.123291
17061    0.121887
17062    0.121085
Name: Diffusion Pseudotime, Length: 17063, dtype: float64

In [13]:
# df = pd.DataFrame(dict)
  
# # show the dataframe
# print(df)
  
# list of values of 'Marks' column
marks_list = pseudo['Diffusion Pseudotime'].tolist()

In [85]:
adatadf = adata.obs

In [87]:
adatadf['barcode_short']

KeyError: 'barcode_short'

In [84]:
k = MyKernel(adata).compute_transition_matrix()
k

Reading `adata.obs['barcode_short']`


KeyError: 'barcode_short'

In [8]:
pseudo.reset_index(level=0, inplace=True) #changes the index to a column

In [9]:
pseudo.head()

,Barcode,Diffusion Pseudotime,Monocle Pseudotime,time
0,AAAGGTAGTTGCAAGG_15806X1,0.080624,7.026657,4
1,AACCACATCCACCTCA_15806X1,0.070802,4.444920,4
2,AAGCGAGAGACCAGAC_15806X1,0.069488,2.045337,4
3,AAGTACCGTTCGTACA_15806X1,0.057792,3.756390,4
4,AAGTCGTTCGTAGGGA_15806X1,0.069608,1.996313,4


In [23]:
bcs = pd.read_csv('/Users/geenaildefonso/Dropbox (VU Basic Sciences)/data/RPM_cell_barcodes.csv', index_col=0, header = 0)
print(bcs.shape)

(19370, 3)


In [ ]:
bcs['barcode_short'] = [i.split('_')[0] for i in bcs['Tumor Cell Barcodes']]

# pseudo['Barcode'] = [str(i.split(":")[1].split('x')[0]) for i in pseudo['Barcode']]

filtered_cells = []
for i in pseudo['Barcode']:
    if i in list(bcs['barcode_short']): 
        filtered_cells.append(True)
    else:
        filtered_cells.append(False)
pseudo['kept'] = filtered_cells
# sc.pl.umap(adata, color = 'kept')

In [ ]:
pseudo

In [ ]:
adata = adata[adata.obs['kept'],:]

In [ ]:
adata.write_h5ad('../int/adata01.h5ad')

In [ ]:
adata = sc.read_h5ad('../int/adata01.h5ad')

In [24]:
adatadm = scv.read('RPM_adata_dynamicalmodeling_7548genes.h5ad')

In [25]:
print(adatadm.shape)

(15257, 7548)


In [ ]:
scv.pl.proportions(adata)
adata

# Run scVelo

We will use the dynamical model from scVelo to estimate the velocities. Please make sure to have at least version 0.2.3 of scVelo installed to make use parallelisation in scv.tl.recover_dynamics. On my laptop, using 8 cores, the below cell takes about 1:30 min to execute.

In [ ]:
tfs = ['SP100', 'FOSL1', 'HES1', 'NFKBIZ', 'RELB', 'EPAS1', 'BCL3', 'REST', 'SP110', 'NFKB2', 'TEAD2', 'HMG20B', 'SIX5',
       'RARG', 'TEAD4', 'ZNF217', 'SP140L', 'SOX18', 'HOXC13', 'STAT6', 'ETV4', 'KLF2', 'MITF', 'NR0B2', 'ASCL1', 'ZBTB7C', 'ELF3',
       'RORC', 'FOXA2', 'ETS2','TOX3', 'XBP1', 'ST18', 'FOXA1', 'OVOL2', 'ZNF664', 'TBX10', 'PROX1', 'ETV6', 'CEBPD', 'TFCP2L1', 'FOXJ3',
       'ZNF407', 'ZNF511','ZNF396', 'RBPJ', 'ZSCAN31', 'HOXB5', 'ZNF3', 'TSHZ2', 'ZBTB16', 'ZNF10', 'FLI1', 'GATA4', 'NR0B1', 'NHLH1',
       'NEUROD6', 'ZNF581', 'TCF15', 'LYAR', 'ISL2', 'OLIG2', 'NEUROD1', 'INSM1', 'PAX5', 'SP6', 'MYT1', 'HES6', 'ZNF24', 'ISL1', 'ZNF397',
       'SOX11', 'ZNF253', 'SMAD4', 'RBP1', 'ONECUT2', 'ZNF711', 'DLX5', 'GRIP1', 'ZNF157', 'ZNF713', 'ZNF136', 'FOXN4', 'PATZ1', 'ZNF491',
       'ZBTB21', 'KLF12', 'ZNF501', 'ZNF785', 'CXXC1', 'ZNF324', 'ZNF764', 'ZBTB18', 'KAT8', 'ZNF334', 'POU4F1', 'ZNF250', 'ZNF132',
       'SALL2', 'DLX6', 'MBD1','SOX1', 'ZFP3', 'ZNF543', 'POU2F1', 'NONO', 'SMAD9', 'ZKSCAN2', 'TCF12', 'VEZF1', 'TOX', 'BHLHE22', 'MTA1',
       'TCF3', 'SCRT2', 'RFX7','NHLH2', 'SCRT1', 'RCOR2', 'PURG', 'TBPL1', 'TCF4', 'EBF1', 'ZNF749', 'NEUROD2', 'ZNF423', 'BACH2', 'GLI1',
       'ZFP64','NKX2-1', 'MYC', 'YAP1', 'POU2F3', 'MYCL', 'MYCN', 'ASCL2', 'AVIL', 'CHAT', 'GFI1B',
      'CHGA','EPCAM', 'Tmed11', 'Rsl24d1','Olfr123', 'Adam20','Srsf2', 'G3bp1', 'Snrpd1', 'Pgk1', 'Rps3', 'Psmd1', 'Orc2', 'Psma2', 'Syncrip', 'Cnbp',
            'Gspt1', 'Ppm1g', 'Eif4e', 'Hnrnpa3', 'Abce1', 'Hnrnpu',  'Ifrd1', 'Kpna2', 'Dhx15', 'Mcm4']

In [ ]:
adata.var_names

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, retain_genes = tfs)
# sc.tl.pca(adata)
# sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30)
# scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

In [ ]:
scv.tl.recover_dynamics(adata)

Once we have the parameters, we can use these to compute the velocities and the velocity graph. The velocity graph is a weighted graph that specifies how likely two cells are to transition into another, given their velocity vectors and relative positions.

In [ ]:
scv.tl.velocity(adata, mode="dynamical")
scv.tl.velocity_graph(adata)

In [ ]:
# scv.pl.velocity_embedding_stream(
#     adata, basis="umap", legend_fontsize=12, title="", smooth=0.8, min_mass=4
# )

scv.pl.velocity_embedding_stream(adata, basis='umap',color=['timepoint'])

In [ ]:
scv.tl.velocity(adata, mode="stochastic")
scv.tl.velocity_graph(adata)

In [ ]:
sc.pp.pca(adata)

In [ ]:
scv.pl.pca(adata, color = 'leiden')

In [ ]:
adata

# Run CellRank

CellRank offers various ways to infuse directionality into single-cell data. Here, the directional information comes from RNA velocity, and we use this information to compute initial & terminal states as well as fate probabilities for the dynamical process of pancreatic development.

# Identify Terminal States

Terminal states can be computed by running the following command:


In [ ]:
cr.pl.circular_projection(
    adata, keys=["timepoint", "kl_divergence"], legend_loc="upper right"
)

In [ ]:
cr.tl.terminal_states(adata, cluster_key="leiden", weight_connectivities=0.2)

In [ ]:
cr.tl.terminal_states(adata, cluster_key="timepoint", weight_connectivities=0.2)

In [ ]:
cr.tl.lineages(adata)

model = cr.ul.models.GAM(adata)

In [ ]:
cr.pl.gene_trends(
    adata,
    model,
    ['Rsl24d1'],
    data_key="Ms",
    time_key="dpt_pseudotime",
    show_progress_bar=False,
)

The most important parameters in the above function are:

estimator: this determines what’s going to behind the scenes to compute the terminal states. Options are cr.tl.estimators.CFLARE (“Clustering and Filtering of Left and Right Eigenvectors”) or cr.tl.estimators.GPCCA (“Generalized Perron Cluster Cluster Analysis, [Reuter et al., 2018] and [Reuter et al., 2019], see also our pyGPCCA implementation). The latter is the default, it computes terminal states by coarse graining the velocity-derived Markov chain into a set of macrostates that represent the slow-time scale dynamics of the process, i.e. it finds the states that you are unlikely to leave again, once you have entered them.

cluster_key: takes a key from adata.obs to retrieve pre-computed cluster labels, i.e. ‘clusters’ or ‘louvain’. These labels are then mapped onto the set of terminal states, to associate a name and a color with each state.

n_states: number of expected terminal states. This parameter is optional - if it’s not provided, this number is estimated from the so-called ‘eigengap heuristic’ of the spectrum of the transition matrix.

method: This is only relevant for the estimator GPCCA. It determines the way in which we compute and sort the real Schur decomposition. The default, krylov, is an iterative procedure that works with sparse matrices which allows the method to scale to very large cell numbers. It relies on the libraries SLEPc and PETSc, which you will have to install separately, see our installation instructions. If your dataset is small (<5k cells), and you don’t want to install these at the moment, use method='brandts' [Brandts, 2002]. The results will be the same, the difference is that brandts works with dense matrices and won’t scale to very large cells numbers.

weight_connectivities: weight given to cell-cell similarities to account for noise in velocity vectors.

When running the above command, CellRank adds a key terminal_states to adata.obs and the result can be plotted as:

In [ ]:
scv.pl.umap(adata, color = 'leiden')

In [ ]:
cr.pl.terminal_states(adata) #, save = 'terminal_states_timepoint_0.2.pdf'

In [ ]:
cr.pl.terminal_states(adata) #, save = 'terminal_states_timepoint_0.2.pdf')

# Identify Initial States

The same sort of analysis can now be repeated for the initial states, only that we use the function cr.tl.initial_states this time:

In [ ]:
cr.tl.initial_states(adata, cluster_key="timepoint")
cr.pl.initial_states(adata, discrete=True) #, save = 'initial_states_timepoint.pdf')

# Computing Fate Maps

Once we know the terminal states, we can compute associated fate maps - for each cell, we ask how likely is the cell to develop towards each of the identified terminal states.

In [ ]:
cr.tl.lineages(adata)
cr.pl.lineages(adata, same_plot=False, save = 'absorption_prob_to_term_states.pdf')

We can aggregate the above into a single, global fate map where we associate each terminal state with color and use the intensity of that color to show the fate of each individual cell:

In [ ]:
cr.pl.lineages(adata, same_plot=True, save = 'absorption_prob_lineages_to_term_states.pdf')

# Directed PAGA

We can further aggregate the individual fate maps into a cluster-level fate map using an adapted version of [Wolf et al., 2019] with directed edges. We first compute scVelo’s latent time with CellRank identified root_key and end_key, which are the probabilities of being an initial or a terminal state, respectively.

In [ ]:
scv.tl.recover_latent_time(
    adata, root_key="initial_states_probs", end_key="terminal_states_probs"
)

Next, we can use the inferred pseudotime along with the initial and terminal states probabilities to compute the directed PAGA.

In [ ]:
scv.tl.paga(
    adata,
    groups="timepoint",
    root_key="initial_states_probs",
    end_key="terminal_states_probs",
    use_time_prior="velocity_pseudotime",
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="paga",
    cluster_key="timepoint",
    basis="umap",
    legend_kwargs={"loc": "top right out"},
    legend_loc="on data",
    node_size_scale=4,
    edge_width_scale=1,
    max_edge_width=4,
    title="Directed PAGA",figsize = (25, 7), save = 'paga_timepoint.pdf'
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="paga_pie",
    cluster_key="timepoint",
    basis="umap",
    legend_kwargs={"loc": "top right out"},
    legend_loc="top left out",
    node_size_scale=4,
    edge_width_scale=1,
    max_edge_width=4,
    title="Directed PAGA Pie",save = 'paga_pie_timepoint.pdf'
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="bar",
    cluster_key="timepoint",
    basis="umap", save = 'paga_bar_timepoint.pdf'
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="violin",
    cluster_key="timepoint",
    basis="umap",
    legend_kwargs={"loc": "top right out"},
    legend_loc="top left out",
    node_size_scale=4,
    edge_width_scale=1,
    max_edge_width=4,
    title="directed PAGA", figsize = (30, 5),save = 'paga_violin_timepoint.pdf'
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="heatmap",
    cluster_key="timepoint",
    basis="umap",
    title="Directed PAGA Heatmap",save = 'paga_heatmap_timepoint.pdf'
)

In [ ]:
cr.pl.cluster_fates(
    adata,
    mode="clustermap",
    cluster_key="timepoint",
    basis="umap",
    title="Directed PAGA Cluster Map",save = 'paga_clustermap_timepoint.pdf'
)

We use pie charts to show cell fates averaged per cluster. Edges between clusters are given by transcriptomic similarity between the clusters, just as in normal PAGA.

# Compute Lineage Drivers

We can compute the driver genes for all or just a subset of lineages. We can also restrict this to some subset of clusters by specifying clusters=... (not shown below). In the resulting dataframe, we also see the p-value, the corrected p-value (q-value) and the 95% confidence interval for the correlation statistic.

In [ ]:
cr.tl.lineage_drivers(adata)

Afterwards, we can plot the top 5 driver genes (based on the correlation), e.g. for the Alpha lineage:

In [ ]:
adata.var['day_7_pval']

In [ ]:
cr.pl.lineage_drivers(adata, lineage="day_7", n_genes=5)

In [ ]:
cr.pl.lineage_drivers(adata, lineage="day_11", n_genes=5)

In [ ]:
cr.pl.lineage_drivers(adata, lineage="day_17", n_genes=5)

In [ ]:
cr.pl.lineage_drivers(adata, lineage="day_21", n_genes=5)

# Gene expression trends

The functions demonstrated above are the main functions of CellRank: computing initial and terminal states and probabilistic fate maps. We can use the computed probabilities now to e.g. smooth gene expression trends along lineages.

Let’s start with a temporal ordering for the cells. To get this, we can compute scVelo’s latent time, as computed previously, or alternatively, we can just use CellRank’s initial states to compute a [Haghverdi et al., 2016].

In [ ]:
# compue DPT, starting from CellRank defined root cell
root_idx = np.where(adata.obs["initial_states"] == "1_2")[0][0]
adata.uns["iroot"] = root_idx
sc.tl.dpt(adata)

scv.pl.scatter(
    adata,
    color=["Clusters", root_idx, "latent_time", "dpt_pseudotime"],
    fontsize=16,
    cmap="viridis",
    perc=[2, 98],
    colorbar=True,
    rescale_color=[0, 1],
    title=["clusters", "root cell", "latent time", "dpt pseudotime"],
)

We can plot dynamics of genes in pseudotime along individual trajectories, defined via the fate maps we computed above.

In [ ]:
model = cr.ul.models.GAM(adata)
cr.pl.gene_trends(
    adata,
    model=model,
    data_key="X",
    genes=['Srsf2', 'G3bp1', 'Snrpd1'],
    ncols=3,
    time_key="latent_time",
    same_plot=True,
    hide_cells=True,
    figsize=(15, 4),
    n_test_points=200,
)

We can also visualize the lineage drivers computed above in a heatmap. Below, we do this for the Alpha lineage, i.e. we smooth gene expression for the putative Alpha-drivers in pseudotime, using as cell-level weights the Alpha fate probabilities. We sort genes according to their peak in pseudotime, thus revealing a cascade of gene expression events.

In [ ]:
cr.pl.heatmap(
    adata,
    model,
    genes=adata.varm['terminal_lineage_drivers']["0_3_corr"].sort_values(ascending=False).index[:100],
    show_absorption_probabilities=True,
    lineages="0_3",
    n_jobs=1,
    backend="loky",
)